In [6]:
import pandas as pd
import requests as rq
import tqdm
import time

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [7]:
page = 0
limit = 10
offset = 0
id_object = 0
http_rq_list_objects = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset}&page={page}&limit={limit}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
http_rq_object = 'https://наш.дом.рф/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/'

In [8]:
req = rq.get(http_rq_list_objects)

In [9]:
json_resp = req.json()

In [10]:
total = req.json().get('data').get('total')
print('Всего записей: ' + str(total))
limit = 1000
count_iter = total//limit
if total%limit > 0: count_iter = count_iter + 1
print('Количество необходимых итераций: ' + str(count_iter))

Всего записей: 10081
Количество необходимых итераций: 11


In [11]:
list_id_objects = []
for page_iter in range(count_iter):
    print('Получаем список объектов со страницы ' + str(page_iter+1), end=': ')
    offset = len(list_id_objects)
    http_rq_list_objects = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?page={page_iter}&offset={offset}&limit={limit}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
    # print('    ' + http_rq_list_objects)
    req_page_iter = rq.get(http_rq_list_objects)
    json_req_page_iter = req_page_iter.json()
    if json_req_page_iter.get('errcode') != str(0):
        print(json_req_page_iter.get('errcode'))
        break
    count_objects = len(json_req_page_iter.get('data').get('list'))
    print('    Объектов на этой странице ' + str(count_objects))
    for iter_objects in range(count_objects):
        list_id_objects.append((json_req_page_iter.get('data').get('list')[iter_objects]).get('objId'))
list_id_objects = list(set(list_id_objects))

Получаем список объектов со страницы 1:     Объектов на этой странице 1000
Получаем список объектов со страницы 2:     Объектов на этой странице 1000
Получаем список объектов со страницы 3:     Объектов на этой странице 1000
Получаем список объектов со страницы 4:     Объектов на этой странице 1000
Получаем список объектов со страницы 5:     Объектов на этой странице 1000
Получаем список объектов со страницы 6:     Объектов на этой странице 1000
Получаем список объектов со страницы 7:     Объектов на этой странице 1000
Получаем список объектов со страницы 8:     Объектов на этой странице 1000
Получаем список объектов со страницы 9:     Объектов на этой странице 1000
Получаем список объектов со страницы 10:     Объектов на этой странице 1000
Получаем список объектов со страницы 11:     Объектов на этой странице 81


In [12]:
objects_data = []
for iter_id_object in tqdm.trange(len(list_id_objects)):
    time.sleep(1/2)
    rq_obj = rq.get(http_rq_object + str(list_id_objects[iter_id_object]))
    if rq_obj.status_code != 200:
        print('Возникла ошибка с кодом [' + str(rq_obj.status_code) + ']. Описание ошибки: [' + str(rq_obj.json().get('errcode')) + ']')
    objects_data.append(rq_obj.json())
len(objects_data)
backup_objects_data = objects_data.copy()

 12%|█▏        | 1188/10077 [18:10<2:16:01,  1.09it/s]


ConnectionError: HTTPSConnectionPool(host='xn--80az8a.xn--d1aqf.xn--p1ai', port=443): Max retries exceeded with url: /%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/36410 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x000002272D0526A0>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера',))

In [301]:
# rq_obj.json()
https://наш.дом.рф/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/37272
objects_data

[{'data': {'address': 'г Набережные Челны, вдоль проспекта Яшьлек, блок-секция 1',
   'conclusion': True,
   'developer': {'buildObjCnt': 2,
    'comissObjCnt': 4,
    'companyGroupId': 156057149,
    'devEmail': 'tatkamstroy@mail.ru',
    'devEmplMainFullNm': 'Мусин Тагир Максимович',
    'devFactAddr': 'Респ Татарстан г Набережные Челны пр-кт Автозаводский д. 59 помещение 1008',
    'devFullCleanNm': 'СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИКТАТКАМСТРОЙ',
    'devId': 6946,
    'devInn': '1650173193',
    'devKpp': '165001001',
    'devLegalAddr': 'Татарстан Республика, город Набережные Челны, проспект Автозаводский дом 59 помещение 1008',
    'devOgrn': '1081650004281',
    'devOrgRegRegionCd': 16,
    'devPhoneNum': '+7(855)544-03-74',
    'devShortCleanNm': 'СЗТКС',
    'devShortNm': 'СЗТКС',
    'devSite': 'www.таткамстрой.рф',
    'developerGroupName': 'ТАТКАМСТРОЙ',
    'fundGuarantyFlg': 0,
    'govFundFlg': 0,
    'lastRpdId': 550231,
    'objGuarantyEscrowFlg': 1,
    'orgForm': {'fullF

In [273]:
list_id_objects[0:1]

[32787]

In [269]:
print(http_rq_object + str(list_id_objects[0]))

https://наш.дом.рф/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/32787


In [271]:
req_id.json()

{'data': {'address': 'г Набережные Челны, вдоль проспекта Яшьлек, блок-секция 1',
  'conclusion': True,
  'developer': {'buildObjCnt': 2,
   'comissObjCnt': 4,
   'companyGroupId': 156057149,
   'devEmail': 'tatkamstroy@mail.ru',
   'devEmplMainFullNm': 'Мусин Тагир Максимович',
   'devFactAddr': 'Респ Татарстан г Набережные Челны пр-кт Автозаводский д. 59 помещение 1008',
   'devFullCleanNm': 'СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИКТАТКАМСТРОЙ',
   'devId': 6946,
   'devInn': '1650173193',
   'devKpp': '165001001',
   'devLegalAddr': 'Татарстан Республика, город Набережные Челны, проспект Автозаводский дом 59 помещение 1008',
   'devOgrn': '1081650004281',
   'devOrgRegRegionCd': 16,
   'devPhoneNum': '+7(855)544-03-74',
   'devShortCleanNm': 'СЗТКС',
   'devShortNm': 'СЗТКС',
   'devSite': 'www.таткамстрой.рф',
   'developerGroupName': 'ТАТКАМСТРОЙ',
   'fundGuarantyFlg': 0,
   'govFundFlg': 0,
   'lastRpdId': 550231,
   'objGuarantyEscrowFlg': 1,
   'orgForm': {'fullForm': 'Общество с огранич

In [209]:
len(list_id_objects)

2000

In [210]:
len(set(list_id_objects))

1000

In [199]:
pd.DataFrame(list_id_objects).nunique

<bound method DataFrame.nunique of            0
0      13399
1      13398
2      13383
3      13397
4      13391
5      23682
6      36321
7      37071
8      36912
9      23494
10     27476
11     27475
12     26194
13     27474
14     27473
15     10891
16     39174
17     39176
18     39175
19     27049
20     41421
21     33928
22     41420
23     30292
24     18363
25     18709
26     18708
27     21535
28     21534
29      9335
...      ...
10970  11657
10971  11095
10972  11302
10973  10415
10974  11638
10975  18764
10976  18756
10977  44757
10978  13669
10979  18758
10980  18754
10981  37139
10982  18763
10983   7196
10984  18762
10985  36533
10986  18761
10987  18759
10988  40756
10989   7198
10990  18753
10991   7199
10992  36707
10993  18760
10994  44060
10995  18755
10996  44762
10997  18750
10998  37072
10999  36460

[11000 rows x 1 columns]>

In [156]:
json_resp.get('errcode')

'0'

In [157]:
json_resp.get('data').get('list')

[{'buildType': 'Нежилое',
  'developer': {'companyGroup': 699421001,
   'devId': 306,
   'devInn': '7701651356',
   'fullName': '2МЕН ГРУПП',
   'groupName': '2МЕН ГРУПП ДЕВЕЛОПМЕНТ',
   'orgForm': {'fullForm': 'Акционерное общество', 'id': 1, 'shortForm': 'АО'},
   'shortName': '2МЕН ГРУПП'},
  'latitude': 57.1519,
  'longitude': 65.5544,
  'objAddr': 'г Тюмень, ул Профсоюзная, д. 56',
  'objCompensFundFlg': 0,
  'objElemLivingCnt': 0,
  'objFloorMax': 1,
  'objFloorMin': 1,
  'objGuarantyEscrowFlg': 0,
  'objId': 13399,
  'objProblemFlg': 0,
  'objReady100PercDt': '2024-12-31',
  'objSquareLiving': 0.0,
  'objStatus': 0,
  'pdId': 28141,
  'problemFlag': 'NONE',
  'residentialBuildings': 0,
  'rpdNum': '72-000097',
  'rpdRegionCd': 72,
  'siteStatus': 'Строящиеся'},
 {'buildType': 'Жилое',
  'developer': {'companyGroup': 699421001,
   'devId': 306,
   'devInn': '7701651356',
   'fullName': '2МЕН ГРУПП',
   'groupName': '2МЕН ГРУПП ДЕВЕЛОПМЕНТ',
   'orgForm': {'fullForm': 'Акционерное

In [54]:
objects_list = json_resp.get('data').get('list')

In [85]:
id_object = objects_list[0].get('objId')
id_object

13399

In [53]:
len(json_resp.get('data').get('list'))
total = json_resp.get('data').get('total')
total
count_iter = 0
count_iter = total//limit
if total%limit > 0: count_iter = count_iter+1
count_iter

11

In [89]:
print(http_rq_object + str(id_object))
req_object = rq.get(http_rq_object + str(id_object))

https://наш.дом.рф/сервисы/api/object/13399


In [170]:
object_data = (req_object.json()).get('data')
object_data

{'address': 'г Тюмень, ул Профсоюзная, д. 56',
 'conclusion': False,
 'developer': {'buildObjCnt': 5,
  'comissObjCnt': 0,
  'companyGroupId': 699421001,
  'devEmail': 'info@2mengroup.ru',
  'devEmplMainFullNm': 'Киселев Сергей Михайлович',
  'devFactAddr': 'Тюменская область, город Тюмень, улица Комсомольская 75/5',
  'devFullCleanNm': '2МЕН ГРУПП',
  'devId': 306,
  'devInn': '7701651356',
  'devKpp': '720301001',
  'devLegalAddr': 'Тюменская область, город Тюмень, улица Комсомольская дом 75/5',
  'devOgrn': '1067746424899',
  'devOrgRegRegionCd': 72,
  'devPhoneNum': '+7(345)279-18-88',
  'devShortCleanNm': '2МЕН ГРУПП',
  'devShortNm': '2МЕН ГРУПП',
  'devSite': '2mengroup.ru',
  'developerGroupName': '2МЕН ГРУПП ДЕВЕЛОПМЕНТ',
  'fundGuarantyFlg': 1,
  'govFundFlg': 0,
  'lastRpdId': 551365,
  'objGuarantyEscrowFlg': 0,
  'orgForm': {'fullForm': 'Акционерное общество', 'id': 1, 'shortForm': 'АО'},
  'problObjCnt': 0,
  'regRegionDesc': 'Тюменская область'},
 'floorMax': 1,
 'floorM

In [173]:
test_dict = {
'id' : object_data.get('id'),
'address' : object_data.get('address'),
'devEvmail' : object_data.get('developer').get('devEvmail'),
'devFactAddr' : object_data.get('developer').get('devFactAddr'),
'devFullCleanNm' : object_data.get('developer').get('devFullCleanNm'),
'devInn' : object_data.get('developer').get('devInn'),
'devKpp' : object_data.get('developer').get('devKpp'),
'devSite' : object_data.get('developer').get('devSite'),
'floorMax' : object_data.get('floorMax'),
'floorMin' : object_data.get('floorMin'),
'generalContractorNm' : object_data.get('generalContractorNm'),
'isAvailableWantLiveHere' : object_data.get('isAvailableWantLiveHere'),
'nonlivFirstFloor' : object_data.get('nonlivFirstFloor'),
'objElemLivingCnt' : object_data.get('objElemLivingCnt'),
'objElemParkingCnt' : object_data.get('objElemParkingCnt'),
'objElevatorCargoCnt' : object_data.get('objElevatorCargoCnt'),
'objElevatorPassengerCnt' : object_data.get('objElevatorPassengerCnt'),
'objElevatorWheelchairCnt' : object_data.get('objElevatorWheelchairCnt'),
'objEnergyEffShortDesc' : object_data.get('objEnergyEffShortDesc'),
'objFlatCnt' : object_data.get('objFlatCnt'),
'objFlatSq' : object_data.get('objFlatSq'),
'objFloorCnt' : object_data.get('objFloorCnt'),
'objGreenHouseFlg' : object_data.get('objGreenHouseFlg'),
'objInfrstrBicycleLaneFlg' : object_data.get('objInfrstrBicycleLaneFlg'),
'objInfrstrCurbLoweringFlg' : object_data.get('objInfrstrCurbLoweringFlg'),
'objInfrstrNotObjPrkngCnt' : object_data.get('objInfrstrNotObjPrkngCnt'),
'objInfrstrObjPrkngCnt' : object_data.get('objInfrstrObjPrkngCnt'),
'objInfrstrPlaygrndCnt' : object_data.get('objInfrstrPlaygrndCnt'),
'objInfrstrRampFlg' : object_data.get('objInfrstrRampFlg'),
'objInfrstrSportGroundCnt' : object_data.get('objInfrstrSportGroundCnt'),
'objInfrstrTrashAreaCnt' : object_data.get('objInfrstrTrashAreaCnt'),
'objLkClassDesc' : object_data.get('objLkClassDesc'),
'objLkFinishTypeDesc' : object_data.get('objLkFinishTypeDesc'),
'objLkFreePlanDesc' : object_data.get('objLkFreePlanDesc'),
'objPriceAvg' : object_data.get('objPriceAvg'),
'objReady100PercDt' : object_data.get('objReady100PercDt'),
'objSquareLiving' : object_data.get('objSquareLiving'),
'objStatus' : object_data.get('objStatus'),
'photoRenderDTO' : object_data.get('photoRenderDTO'),
'soldOutPerc' : object_data.get('soldOutPerc'),
'wallMaterialShortDesc' : object_data.get('wallMaterialShortDesc'),
}

In [174]:
pd.DataFrame(test_dict)

,address,devEvmail,devFactAddr,devFullCleanNm,devInn,devKpp,devSite,floorMax,floorMin,generalContractorNm,id,isAvailableWantLiveHere,nonlivFirstFloor,objElemLivingCnt,objElemParkingCnt,objElevatorCargoCnt,objElevatorPassengerCnt,objElevatorWheelchairCnt,objEnergyEffShortDesc,objFlatCnt,objFlatSq,objFloorCnt,objGreenHouseFlg,objInfrstrBicycleLaneFlg,objInfrstrCurbLoweringFlg,objInfrstrNotObjPrkngCnt,objInfrstrObjPrkngCnt,objInfrstrPlaygrndCnt,objInfrstrRampFlg,objInfrstrSportGroundCnt,objInfrstrTrashAreaCnt,objLkClassDesc,objLkFinishTypeDesc,objLkFreePlanDesc,objPriceAvg,objReady100PercDt,objSquareLiving,objStatus,photoRenderDTO,soldOutPerc,wallMaterialShortDesc
0,"г Тюмень, ул Профсоюзная, д. 56",None,"Тюменская область, город Тюмень, улица Комсомольская 75/5",2МЕН ГРУПП,7701651356,720301001,2mengroup.ru,1,1,ООО Транспромжилстрой-2005 (ИНН: 7204097730),13399,True,1,0,125,7,8,0,B,0,0.0,1,0,0,1,0,0,0,1,0,1,Элитный,Без отделки,Нет,50768,2024-12-31,0.0,0,"{'objId': 13399, 'pdId': 28141, 'rpdId': 551292, 'devId': 306, 'ordNum': 1, 'objRenderPhotoSize': 996519, 'objRenderPhotoUrl': 'https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/file/0b989b49-d32a-40e...",0.917243,Другое


In [101]:
set(object_data.keys())

{'address',
 'conclusion',
 'developer',
 'floorMax',
 'floorMin',
 'generalContractorNm',
 'id',
 'isAvailableWantLiveHere',
 'loadDttm',
 'miniUrl',
 'newBuildingId',
 'nonlivFirstFloor',
 'objCompensFundFlg',
 'objElemLivingCnt',
 'objElemParkingCnt',
 'objElevatorCargoCnt',
 'objElevatorPassengerCnt',
 'objElevatorWheelchairCnt',
 'objEnergyEffShortDesc',
 'objFlatCnt',
 'objFlatSq',
 'objFloorCnt',
 'objGreenHouseFlg',
 'objGuarantyEscrowFlg',
 'objInfrstrBicycleLaneFlg',
 'objInfrstrCurbLoweringFlg',
 'objInfrstrNotObjPrkngCnt',
 'objInfrstrObjPrkngCnt',
 'objInfrstrPlaygrndCnt',
 'objInfrstrRampFlg',
 'objInfrstrSportGroundCnt',
 'objInfrstrTrashAreaCnt',
 'objLkClassDesc',
 'objLkFinishTypeDesc',
 'objLkFreePlanDesc',
 'objLkLatitude',
 'objLkLongitude',
 'objLkSaleDepEmail',
 'objNonlivElemCnt',
 'objPriceAvg',
 'objProblemFlg',
 'objReady100PercDt',
 'objSquareLiving',
 'objStatus',
 'objTransferPlanDt',
 'objectTransportInfo',
 'objectType',
 'pdId',
 'photoRenderDTO',
 'reg